In [1]:
import asyncio
from dotenv import load_dotenv
import aiohttp

load_dotenv()
import nest_asyncio

nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
from suno import SongsGen
from concurrent.futures import ThreadPoolExecutor

In [2]:
import json
# Load json from data/data.json

with open('data/data.json') as f:
    data = json.load(f)
print(data['music_description'])

Upbeat, quirky instrumental music with a 'mind-blown' sound effect when the hash table is first mentioned


In [3]:
async def async_generate_music(text):
    i = SongsGen(os.environ.get("SUNO_COOKIE"))
    print(i.get_limit_left())
    loop = asyncio.get_running_loop()

    result = None
    # Use a ThreadPoolExecutor to run synchronous functions in threads
    with ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool,
            lambda: i.get_songs(text, make_instrumental=True),
        )

    if not result:
        return None

    link = result["song_url"]

    # Ensure the session is properly closed using async with
    async with aiohttp.ClientSession() as session:
        async with session.get(link) as response:
            if response.status == 200:
                return await response.read()

    return None

In [4]:
result = asyncio.run(async_generate_music(data['music_description']))

# Save result as mp3
with open('data/music.mp3', 'wb') as f:
    f.write(result)
    f.close()

197


Waiting for results...

.

.

.

...

.

.

.

.

.